# Flight Delay Prediction Part 2 - Weather Data

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Import file handling from google.colab 
from google.colab import files

In [ ]:
# Getting the required Weather Data JSON files
import pandas as pd
import numpy as np
import csv
import json
# Path for files in Drive, change as required
path = "/content/gdrive/My Drive/Flight Delay/Data/Weather/"
months = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
years = ["2016-", "2017-"]
airports = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", 
            "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]
jsondf = []
# List of Pandas Dataframes created for JSON files
for airport in airports:
  for year in years:
    for month in months:
      jsondf.append(pd.read_json(path+airport+"/"+year+month+".json"))

In [ ]:
# Combine all Dataframes into one - only date, hourly from weather required
templ = []
for _ in jsondf:
  df = pd.DataFrame(_["data"]["weather"])
  df = df[["date", "hourly"]]
  templ.append(df)
wtempdf = pd.concat(templ)
# Re-indexing
wtempdf.reset_index()
wtempdf.index = range(10965)
wtempdf["i"] = wtempdf.index
# Adding airport column
wtempdf["airport"] = wtempdf["i"].apply(lambda x: airports[x//731])
del wtempdf["i"]

In [ ]:
# Splitting dates into individual hours
rows = []
_ = wtempdf.apply(lambda row: [rows.append([row['date'], h, row['airport']]) 
                         for h in row.hourly], axis=1)
wdf2 = pd.DataFrame(rows, columns=wtempdf.columns)
# Adding hour column
wdf2["hour"] = wdf2.index
wdf2["hour"] = wdf2["hour"]%24

In [ ]:
# Split hourly into individual columns for each weather aspect
wdf2 = pd.concat([wdf2.drop(['hourly'], axis=1), wdf2['hourly'].apply(pd.Series)], axis=1)

In [ ]:
# Select required weather features to retain, and other required columns
wdf2 = wdf2[["date","hour","airport","tempC","windspeedKmph","winddir16Point","weatherCode","precipMM","humidity","pressure","cloudcover","visibility","DewPointC","WindChillC","WindGustKmph"]]

In [ ]:
# Re-indexing by creating a unique code for each row using date, hour and airport
def code(row):
  return row["date"]+str(row["hour"])+row["airport"]
wdf2["index"] = wdf2.apply(lambda row: code(row), axis = 1)
wdf2.index = wdf2["index"]

In [ ]:
# Delete extra index column
del wdf2["index"]
# View DataFrame
wdf2

In [ ]:
# Loading FLIGHTSDF.csv to combine weather data
fddf = pd.read_csv("/content/gdrive/My Drive/Flight Delay/Data/FLIGHTSDF.csv")
# Re-indexing fddf with similar code for easy joining
def codef(row):
  return row["FL_DATE"]+str(row["HOUR"])+row["ORIGIN"]
fddf["index"] = fddf.apply(lambda row: codef(row), axis = 1)

In [ ]:
# Re-indexing and delete unwanted column
fddf.index = fddf["index"]
del fddf["Unnamed: 0"]
del fddf["index"]
# View DataFrame
fddf

In [ ]:
# Join as per conditions and view result
result = pd.concat([fddf, wdf2], axis=1, join_axes=[fddf.index])
result

In [ ]:
# Create CSV for result and download it
result.to_csv("RESULT.csv")
files.download("RESULT.csv")

In [ ]:
# DataFrames with delay values less than, equal to and more than 0
a, b, c = result[result["DEP_DELAY"] < 0], result[result["DEP_DELAY"] == 0], result[result["DEP_DELAY"] > 0]

In [ ]:
# Plotting the number of values in each category
s = pd.Series([len(a), len(b), len(c)])
s.index = ["Delay < 0","Delay = 0","Delay > 0"]
get = s.plot(title="Distribution", kind="pie")
get.set_ylabel("")